# 
Yolo8云端部署
#### 2025.2.17
## 过程说明:
1. 分别载入模型和数据集
2. 整体迁移至工程文件夹下
3. 运行训练脚本
4. [可选]运行推理，结果保存在/kaggle/working/pred_rst.jpg文件
5. 转存训练好的权重文件至/kaggle/working/train.zip
## 注意:
1. 开启Kaggle平台的GPU（P100）后，运行nvidia --smi 可以看到该GPU支持CUDA12.6版本。而且note环境里已经预装了CUDA11.2、torch2.5.1+cu121
2. 运行前检查：模型文件位置、数据集位置、数据集配置文件内容

In [1]:
# 工程准备
print("复制工程到根目录")
!cp -r /kaggle/input/yolo8/pytorch/1.0/11/yolo8 /yolo8
!ls /yolo8 -a
print("安装依赖")
!pip install -e /yolo8/
# 数据集准备
print("转移数据库")
!cp -r /kaggle/input/ocean3 /yolo8/datasets
#!cp -r /kaggle/input/river-floating-trash-datasets/datasets /yolo8/datasets
!ls /yolo8/datasets -a

# 查看GPU是否存在：最高支持CUDA12.6
import torch
print("[0]GPU:")
def check_gpu_availability():
    # 使用 torch.cuda.is_available() 函数检测 GPU 是否可用
    if torch.cuda.is_available():
        # 获取可用 GPU 的数量
        num_gpus = torch.cuda.device_count()
        print(f"GPU 可用，可用的 GPU 数量为: {num_gpus}")
        # 遍历每个可用的 GPU 并打印其名称
        for i in range(num_gpus):
            gpu_name = torch.cuda.get_device_name(i)
            print(f"GPU {i}: {gpu_name}")
    else:
        print("GPU 不可用，将使用 CPU 进行计算。")
check_gpu_availability()
# 查看CUDA版本12.2
print("[1]CUDA:")
!nvcc --version
# 查看pytorch版本2.5.1
print("[2]torch:"+torch.__version__)

# 查看python版本3.10
print("[3]Python:")
!python --version
# 测试已通过
print("准备完成")

复制工程到根目录
.		 docs	      noteB		       task
..		 examples     noteB.ipynb	       tests
best.pt		 .git	      .pre-commit-config.yaml  ultralytics
CITATION.cff	 .gitignore   README.md		       ultralytics.egg-info
CONTRIBUTING.md  LICENSE      README.zh-CN.md	       yolov8n.pt
datasets	 MANIFEST.in  requirements.txt
default.jpg	 mkdocs.yml   setup.cfg
docker		 none.txt     setup.py
安装依赖
Obtaining file:///yolo8
  Preparing metadata (setup.py) ... done
  Running setup.py develop for ultralytics
转移数据库
.  ..  oc_cld.yaml  ocean3  oc_loc.yaml
[0]GPU:
GPU 可用，可用的 GPU 数量为: 1
GPU 0: Tesla P100-PCIE-16GB
[1]CUDA:
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
[2]torch:2.5.1+cu121
[3]Python:
Python 3.10.12
准备完成


In [2]:
ls /yolo8/datasets/ocean3/ocean/labels

test/  train/  valid/


In [3]:
cd /yolo8/datasets/ocean3/ocean/labels

/yolo8/datasets/ocean3/ocean/labels


In [4]:
ls ./train -a

./         11782.txt  13564.txt  2873.txt  4655.txt  6437.txt  8219.txt
../        11783.txt  13565.txt  2874.txt  4656.txt  6438.txt  821.txt
10000.txt  11784.txt  13566.txt  2875.txt  4657.txt  6439.txt  8220.txt
10001.txt  11785.txt  13567.txt  2876.txt  4658.txt  643.txt   8221.txt
10002.txt  11786.txt  13568.txt  2877.txt  4659.txt  6440.txt  8222.txt
10003.txt  11787.txt  13569.txt  2878.txt  465.txt   6441.txt  8223.txt
10004.txt  11788.txt  1356.txt   2879.txt  4660.txt  6442.txt  8224.txt
10005.txt  11789.txt  13570.txt  287.txt   4661.txt  6443.txt  8225.txt
10006.txt  1178.txt   13571.txt  2880.txt  4662.txt  6444.txt  8226.txt
10007.txt  11790.txt  13572.txt  2881.txt  4663.txt  6445.txt  8227.txt
10008.txt  11791.txt  13573.txt  2882.txt  4664.txt  6446.txt  8228.txt
10009.txt  11792.txt  13574.txt  2883.txt  4665.txt  6447.txt  8229.txt
1000.txt   11793.txt  13575.txt  2884.txt  4666.txt  6448.txt  822.txt
10010.txt  11794.txt  13576.txt  2885.txt  4667.txt  6449.txt  823

In [5]:
cd /yolo8

/yolo8


In [6]:
cat ./datasets/ocean3/ocean/oc_cld.yaml

path: /yolo8/datasets
train: ./ocean3/ocean/images/train
val: ./ocean3/ocean/images/valid
test: ./ocean3/ocean/images/test

nc: 3
names: ['Plants', 'other', 'Plastic']

In [7]:
ls ./ -a

./               docs/        noteB                    task/
../              examples/    noteB.ipynb              tests/
best.pt          .git/        .pre-commit-config.yaml  ultralytics/
CITATION.cff     .gitignore   README.md                ultralytics.egg-info/
CONTRIBUTING.md  LICENSE      README.zh-CN.md          yolov8n.pt
datasets/        MANIFEST.in  requirements.txt
default.jpg      mkdocs.yml   setup.cfg
docker/          none.txt     setup.py


In [8]:
# 开始推理

# 所有相对路径基于/yolo8


# 0训练    1推理
way = 0


def train():
    from ultralytics import YOLO
    # Load a model
    model = YOLO("yolov8n.yaml")  # build a new model from scratch
    #model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

    # Use the model
    results = model.train(data="./datasets/oc_cld.yaml", \
                          epochs=10,\
                          imgsz=640,\
                          batch=32,\
                          device=0)  # 0=GPU

def pred(image_path,rst_path):
    from ultralytics import YOLO
    import cv2
    import yaml

    # 加载训练好的权重
    model = YOLO('./runs/detect/train/weights/best.pt')
    results = model.predict(source=image_path)

    # 读取原始图像
    image = cv2.imread(image_path)
    cv2.imwrite('/kaggle/working/pred_src.jpg',image)

    # 处理推理结果
    for result in results:
        boxes = result.boxes  # 获取边界框信息
        if boxes is not None:
            for box in boxes:
                # 获取边界框坐标
                xyxy = box.xyxy[0].cpu().numpy().astype(int)
                x1, y1, x2, y2 = xyxy

                # 获取类别索引和置信度
                cls = int(box.cls[0])
                conf = float(box.conf[0])

                with open('./datasets/RFT.yaml', 'r') as f:
                    data = yaml.safe_load(f)
                names = data['names']

                # 绘制边界框
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

                # 准备标签文本
                label = f'{names[cls]}: {conf:.2f}'

                # 绘制标签背景
                (label_width, label_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
                cv2.rectangle(image, (x1, y1 - label_height - 5), (x1 + label_width, y1), (0, 255, 0), -1)

                # 绘制标签文本
                cv2.putText(image, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    # 保存带有检测结果的图像
    return cv2.imwrite(rst_path, image)

if way==0:
    train()
# 训练结果转存
    !zip -r /kaggle/working/train.zip /yolo8/runs/detect/train
else:
    rstImg_path = '/kaggle/working/pred_rst.jpg'
    # 进行单张图像推理  6,15,19,22,31,35,40,43,46,72,73,78,80,85,88,92
    rst = pred(image_path='./datasets/RFT/images/test/000006.jpg',rst_path=rstImg_path)
    if rst:
        print(f"保存成功：{rstImg_path}")
    else:
        print("出错")


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.Conv                  [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.Conv                  [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.C2f                   [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.Conv                  [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.C2f                   [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.Conv                  [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.C2f                   [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.Conv                  [128

  adding: yolo8/runs/detect/train/ (stored 0%)
  adding: yolo8/runs/detect/train/val_batch1_pred.jpg (deflated 9%)
  adding: yolo8/runs/detect/train/train_batch2.jpg (deflated 14%)
  adding: yolo8/runs/detect/train/val_batch2_pred.jpg (deflated 15%)
  adding: yolo8/runs/detect/train/val_batch2_labels.jpg (deflated 15%)
  adding: yolo8/runs/detect/train/P_curve.png (deflated 8%)
  adding: yolo8/runs/detect/train/weights/ (stored 0%)
  adding: yolo8/runs/detect/train/weights/best.pt (deflated 11%)
  adding: yolo8/runs/detect/train/weights/last.pt (deflated 11%)
  adding: yolo8/runs/detect/train/F1_curve.png (deflated 10%)
  adding: yolo8/runs/detect/train/results.csv (deflated 82%)
  adding: yolo8/runs/detect/train/val_batch1_labels.jpg (deflated 9%)
  adding: yolo8/runs/detect/train/args.yaml (deflated 51%)
  adding: yolo8/runs/detect/train/results.png (deflated 10%)
  adding: yolo8/runs/detect/train/R_curve.png (deflated 11%)
  adding: yolo8/runs/detect/train/val_batch0_pred.jpg (defla

In [9]:
ls -a /yolo8/runs/detect/train

./                                                train_batch0.jpg
../                                               train_batch1.jpg
args.yaml                                         train_batch2.jpg
confusion_matrix.png                              val_batch0_labels.jpg
events.out.tfevents.1744623445.b8759839dc6f.18.0  val_batch0_pred.jpg
F1_curve.png                                      val_batch1_labels.jpg
P_curve.png                                       val_batch1_pred.jpg
PR_curve.png                                      val_batch2_labels.jpg
R_curve.png                                       val_batch2_pred.jpg
results.csv                                       weights/
results.png
